# Demonstrating Agentic Capabilities with DeepSeek Thinking Models

This notebook demonstrates how to use DeepSeek thinking models via OpenRouter to showcase agentic capabilities without using complex frameworks. We'll see how LLMs can:

1. **Plan** - Create a structured approach to solve problems
2. **Reason** - Think through intermediate steps logically
3. **Reflect** - Evaluate their own work and identify issues
4. **Revise** - Make changes based on reflection
5. **Produce** - Generate final outputs

## What are Thinking Models?

Traditional LLMs generate responses directly, giving you only the final answer. **Thinking models** (also called reasoning models) show their "thought process" - the internal reasoning steps they take to arrive at an answer. This transparency helps us:

- Understand how the model solved a problem
- Verify the logic and catch potential errors
- Learn from the model's reasoning approach
- Build more reliable AI systems

## Why This Matters for AI Agents

AI agents need to be more than just question-answering systems. They need to:
- Break down complex problems into steps
- Plan their approach before acting
- Reflect on their work and self-correct
- Show their reasoning for accountability

DeepSeek's thinking models demonstrate these capabilities naturally, making them excellent for learning about agentic AI.

## Setup

First, let's set up our environment by loading the OpenRouter API credentials.

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json

# Load environment variables from .env file
load_dotenv("../../.env")

# Initialize OpenAI client with OpenRouter configuration
client = OpenAI(
    base_url=os.getenv("OPENROUTER_BASE_URL"),
    api_key=os.getenv("OPENROUTER_API_KEY")
)

# Check if credentials are loaded
print(f"API Base URL: {os.getenv('OPENROUTER_BASE_URL')}")
print(f"API Key: {'Loaded successfully' if os.getenv('OPENROUTER_API_KEY') else 'Not found'}")

## Available DeepSeek Models

DeepSeek offers several models with thinking/reasoning capabilities through OpenRouter:

### Model Comparison:

| Model | Parameters | Best For | Cost | Key Features |
|-------|------------|----------|------|--------------|
| **DeepSeek R1** | 671B (37B active) | Complex reasoning, production use | \$\$\$ | Full reasoning capabilities, highest quality |
| **DeepSeek R1 Distill Qwen 32B** | 32B | Balanced performance/cost | \$\$ | Good reasoning, faster than R1 |
| **DeepSeek R1 Distill Llama 70B** | 70B | High quality, open-source compatible | \$\$ | Strong reasoning, Llama architecture |
| **DeepSeek Chat** | Varies | General conversation | \$ | No reasoning visibility, fast |

For this notebook, we'll use the paid DeepSeek R1 for best results, but you can experiment with others.

Let's set up our helper functions to work with these models and correctly access their reasoning capabilities.

In [ ]:
def call_deepseek_thinking_model(prompt, model="deepseek/deepseek-r1"):
    """Call a DeepSeek thinking-enabled model with the OpenRouter API.
    
    This function accesses the reasoning capabilities of DeepSeek models
    through OpenRouter by using the include_reasoning parameter.
    """
    try:
        # Using the include_reasoning parameter to access the model's thinking process
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            extra_body={
                "include_reasoning": True  # This is the key parameter to get reasoning
            }
        )
        
        # Extract the reasoning (thinking) process
        thinking = "No explicit thinking process detected"
        answer = response.choices[0].message.content
        
        # Check if there's an extra field containing reasoning
        if hasattr(response, 'choices') and hasattr(response.choices[0].message, 'reasoning'):
            thinking = response.choices[0].message.reasoning
        # Fall back to the older format for reasoning_content
        elif hasattr(response, 'choices') and hasattr(response.choices[0].message, 'reasoning_content'):
            thinking = response.choices[0].message.reasoning_content
        # If reasoning is not in response fields, try to extract from the <think> tags in the content
        elif "<think>" in answer and "</think>" in answer:
            thinking_start = answer.find("<think>") + 7
            thinking_end = answer.find("</think>")
            thinking = answer[thinking_start:thinking_end].strip()
            # Get the final answer (after </think>)
            answer = answer[thinking_end + 8:].strip()  # +8 to skip the closing tag
            
        return {
            "thinking": thinking,
            "answer": answer,
            "raw_response": response
        }
    except Exception as e:
        return {"error": str(e)}

def call_regular_model(prompt, model="deepseek/deepseek-chat"):
    """Call a regular model without thinking capabilities."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        
        return {
            "answer": response.choices[0].message.content
        }
    except Exception as e:
        return {"error": str(e)}

def display_thinking_response(result):
    """Pretty print the thinking and response from a model."""
    if "error" in result:
        print(f"Error: {result['error']}")
        return
        
    print("\n" + "=" * 80)
    print("🧠 THINKING PROCESS:")
    print("=" * 80)
    print(result["thinking"] if result["thinking"] else "No explicit thinking process detected")
    
    print("\n" + "=" * 80)
    print("✨ FINAL RESPONSE:")
    print("=" * 80)
    print(result["answer"])
    
    # Optional: Print the raw response for debugging
    if "debug" in globals() and debug:
        print("\n" + "=" * 80)
        print("🔍 RAW RESPONSE:")
        print("=" * 80)
        print(result.get("raw_response", "No raw response available"))

## Note on OpenRouter Parameters

OpenRouter supports different parameter formats for accessing reasoning capabilities:
- `include_reasoning: True` - The format we'll use in this notebook
- `reasoning: {}` - An alternative unified parameter format

Both work similarly. We'll use `include_reasoning` for consistency.

In [ ]:
# Set to True to see raw responses (helpful for debugging)
debug = False

## Example 1: Basic Planning and Reasoning

Let's start with a simple example that demonstrates basic planning and reasoning - solving a word problem.

In [ ]:
simple_prompt = """
A farmer has 17 sheep, all but 9 die. How many sheep are left? 
"""

# First let's see how a regular model responds
regular_result = call_regular_model(simple_prompt)
print("REGULAR MODEL RESPONSE:")
print("=" * 80)
print(regular_result["answer"])

# Now let's see the thinking model's response
thinking_result = call_deepseek_thinking_model(simple_prompt)
display_thinking_response(thinking_result)

## Example 2: Multi-Step Reasoning with Reflection

Now let's try a more complex problem that requires multi-step reasoning and reflection.

In [ ]:
complex_prompt = """
Alice, Bob, and Charlie are playing a game with marbles. 
- Alice starts with twice as many marbles as Bob. 
- Bob starts with 3 times as many marbles as Charlie. 
- Charlie starts with 5 marbles.
- After the first round, Alice loses half her marbles to Bob.
- After the second round, Bob gives 1/3 of his marbles to Charlie.

How many marbles does each person have at the end? Show your reasoning step by step.
"""

thinking_result = call_deepseek_thinking_model(complex_prompt)
display_thinking_response(thinking_result)

## Example 3: Creativity with Planning and Self-Critique

Let's now see how the model handles a creative task that requires planning and then critiquing its own work.

In [ ]:
creative_prompt = """
Task: Create a short story (about 150 words) about an astronaut discovering something unexpected on Mars.

Please follow these steps:
1. Plan the key elements of the story (characters, setting, conflict, resolution)
2. Write a first draft
3. Review and critique your draft, identifying at least 2 aspects to improve
4. Create a revised, final version of the story based on your critique

Make sure the final story is engaging and contains an interesting twist.
"""

thinking_result = call_deepseek_thinking_model(creative_prompt)
display_thinking_response(thinking_result)

## Example 4: Emulating an Agent for Problem Solving

Now, let's emulate a more complex agent-like behavior where the model needs to break down a problem, research information (simulated), and iteratively improve its solution.

In [ ]:
agent_prompt = """
You are an AI agent tasked with planning a 3-day itinerary for a family visiting San Francisco. 

The family has the following requirements:
- They have two children (ages 8 and 12)
- They want a mix of tourist attractions and local experiences
- Their budget is moderate (not luxury, but not extremely budget-conscious)
- They will be staying near Union Square
- One family member has mild mobility issues and can't walk long distances

Please act as an agent that:
1. First identifies the key constraints and goals of this task
2. Brainstorms potentially suitable attractions and activities
3. Researches and evaluates options (you can simulate this research process)
4. Organizes them into a coherent 3-day itinerary
5. Reflects on the itinerary, identifies potential problems or improvements
6. Revises the itinerary based on your reflection
7. Presents the final itinerary in a user-friendly format

For each step in your process, explain your reasoning and how it contributes to the final solution.
"""

thinking_result = call_deepseek_thinking_model(agent_prompt)
display_thinking_response(thinking_result)

## Example 5: Multi-Agent Simulation

Finally, let's demonstrate a more complex scenario where the model simulates multiple agents with different roles working together to solve a problem.

In [ ]:
multi_agent_prompt = """
Simulate a dialogue between three specialized AI agents collaborating to design a sustainable smart home system:

1. EngineeringAgent: Expert in technical feasibility, energy systems, and IoT infrastructure
2. SustainabilityAgent: Expert in environmental impact, resource efficiency, and sustainable materials
3. UserExperienceAgent: Expert in human-centered design, accessibility, and usability

Their goal is to design a comprehensive smart home system that is technically sound, environmentally sustainable, and user-friendly.

The dialogue should include:
- Each agent introducing their initial ideas from their area of expertise
- Agents identifying conflicts between their individual priorities
- Collaborative problem-solving to resolve these conflicts
- Iterations of the design based on feedback from other agents
- A final consensus design that balances all three perspectives

For each turn in the dialogue, each agent should explicitly reference their specialized knowledge and reasoning process.
"""

thinking_result = call_deepseek_thinking_model(multi_agent_prompt)
display_thinking_response(thinking_result)

## Trying with Different DeepSeek Distilled Models

DeepSeek provides several distilled models that offer a good balance between performance and cost. Let's try a couple of them on a more complex reasoning task.

In [ ]:
# A problem involving logic and reasoning
logic_puzzle = """
Five friends (Alex, Brook, Charlie, Dana, and Evan) are sitting in a row of five chairs, 
numbered 1 to 5 from left to right. We know that:
- Alex is sitting in chair 1 or chair 5.
- Brook is sitting in an even-numbered chair.
- Charlie and Dana are sitting next to each other.
- Evan is not sitting next to Alex or Brook.

Determine who is sitting in each chair.
"""

# Try with a distilled model if available
try:
    distilled_model_result = call_deepseek_thinking_model(logic_puzzle, model="deepseek/deepseek-r1-distill-qwen-14b")
    print("\nUSING DEEPSEEK R1 DISTILL QWEN 14B MODEL:\n")
    display_thinking_response(distilled_model_result)
except Exception as e:
    print(f"Error with distilled model: {e}")
    print("Falling back to free model...")
    fallback_result = call_deepseek_thinking_model(logic_puzzle)
    display_thinking_response(fallback_result)

## Direct Comparison of Models

Let's compare different DeepSeek models on the same challenging math problem to see how their thinking styles differ.

In [ ]:
# A challenging algebra problem
math_problem = """
If the sum of two numbers is 15 and their product is 56, 
what is the sum of their cubes?
"""

# Regular model (no thinking)
regular_result = call_regular_model(math_problem)
print("REGULAR MODEL (NO THINKING):\n")
print(regular_result["answer"])

# DeepSeek R1 (paid model with best reasoning)
deepseek_r1_result = call_deepseek_thinking_model(math_problem, model="deepseek/deepseek-r1")
print("\n" + "=" * 100)
print("DEEPSEEK R1 MODEL (WITH THINKING):")
print("=" * 100)
display_thinking_response(deepseek_r1_result)

# Optional: Try a distilled model for comparison
try:
    distilled_result = call_deepseek_thinking_model(math_problem, model="deepseek/deepseek-r1-distill-qwen-32b")
    print("\n" + "=" * 100)
    print("DEEPSEEK R1 DISTILL (32B) MODEL:")
    print("=" * 100)
    display_thinking_response(distilled_result)
except Exception as e:
    print(f"\nSkipping distilled model: {e}")

## Conclusion

These examples demonstrate how DeepSeek's thinking models can exhibit agentic capabilities - planning, reasoning, reflecting, revising, and producing outputs - all within a single prompt. 

Key observations:

1. **Transparency**: OpenRouter allows us to access the model's reasoning process, giving us insight into how it arrived at its conclusions.

2. **Self-correction**: The thinking models can identify mistakes and make corrections during their reasoning process.

3. **Structured thinking**: The models naturally break down complex problems into manageable steps.

4. **Adaptability**: These models can apply reasoning capabilities to various tasks, from mathematical problems to creative writing to complex planning.

5. **Cost-effectiveness**: By using DeepSeek's models, particularly the free tier or the distilled versions, we can demonstrate agentic capabilities at a fraction of the cost of the largest frontier models.

This approach offers a simple yet powerful way to introduce the concept of AI agents without the complexity of frameworks, focusing instead on the fundamental reasoning capabilities that make agentic behavior possible.

## Extension: Planning & Reasoning Approaches Comparison

This extension demonstrates three different planning and reasoning approaches using DeepSeek models, showing how different prompting techniques can be used to guide model thought processes in different ways.

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt

# Helper function to display thinking process and answer
def display_comparison_result(title, result):
    print("\n" + "=" * 80)
    print(f"🔍 {title}")
    print("=" * 80)
    
    if "error" in result:
        print(f"Error: {result['error']}")
        return
        
    # Display thinking process
    print("\n📝 THINKING PROCESS:")
    print("-" * 40)
    print(result["thinking"] if result["thinking"] else "No explicit thinking process detected")
    
    # Display answer
    print("\n🎯 ANSWER:")
    print("-" * 40)
    print(result["answer"])
    
    # Display metrics if available
    if "metrics" in result:
        print("\n📊 METRICS:")
        print("-" * 40)
        for key, value in result["metrics"].items():
            print(f"{key}: {value}")

In [ ]:
def call_cot_model(problem, model="deepseek/deepseek-r1:free"):
    """Call model with Chain of Thought prompting."""
    prompt = f"""
    {problem}
    
    Let's solve this step-by-step to ensure we get the correct answer.
    """
    
    start_time = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            extra_body={
                "include_reasoning": True
            }
        )
        
        end_time = time.time()
        
        # Extract thinking and response
        thinking = response.choices[0].message.reasoning if hasattr(response.choices[0].message, "reasoning") else ""
        answer = response.choices[0].message.content
        
        # Calculate metrics
        metrics = {
            "response_time": round(end_time - start_time, 2),
            "thinking_length": len(thinking) if thinking else 0,
            "answer_length": len(answer)
        }
        
        return {
            "thinking": thinking,
            "answer": answer,
            "metrics": metrics
        }
    except Exception as e:
        return {"error": str(e)}

In [ ]:
def call_react_model(problem, model="deepseek/deepseek-r1:free"):
    """Call model with ReAct (Reasoning + Acting) prompting."""
    prompt = f"""
    {problem}
    
    Let's solve this by following the ReAct approach (Reasoning + Acting):
    
    Step 1: Think about what information we need
    Step 2: Define the actions we need to take
    Step 3: Take each action, one at a time
    Step 4: Observe the results
    Step 5: Reason about the next step based on observations
    
    When solving, clearly indicate each Thought, Action, and Observation step.
    """
    
    start_time = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            extra_body={
                "include_reasoning": True
            }
        )
        
        end_time = time.time()
        
        # Extract thinking and response
        thinking = response.choices[0].message.reasoning if hasattr(response.choices[0].message, "reasoning") else ""
        answer = response.choices[0].message.content
        
        # Calculate metrics
        metrics = {
            "response_time": round(end_time - start_time, 2),
            "thinking_length": len(thinking) if thinking else 0,
            "answer_length": len(answer),
            "action_count": answer.lower().count("action:") if answer else 0
        }
        
        return {
            "thinking": thinking,
            "answer": answer,
            "metrics": metrics
        }
    except Exception as e:
        return {"error": str(e)}

In [ ]:
def call_tot_model(problem, model="deepseek/deepseek-r1"):
    """Call model with Tree of Thoughts prompting."""
    prompt = f"""
    IMPORTANT: Focus only on solving the following dinner party planning problem. Do not generate any other content.
    
    {problem}
    
    Solve this using the Tree of Thoughts approach:
    
    1. First, generate 3 different initial approaches to solve this problem
    2. For each approach, evaluate its potential success (High/Medium/Low)
    3. Choose the most promising approach and develop it further
    4. If you encounter difficulties, backtrack and try another approach
    5. Continue until you find the solution
    
    Present your thought process as a tree structure with different branches of reasoning.
    Focus entirely on the dinner party planning constraints and provide a practical solution.
    """
    
    start_time = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
            extra_body={
                "include_reasoning": True
            }
        )
        
        end_time = time.time()
        
        # Extract thinking and response
        thinking = ""
        answer = response.choices[0].message.content
        
        # Try to extract reasoning from different possible locations
        if hasattr(response.choices[0].message, 'reasoning'):
            thinking = response.choices[0].message.reasoning
        elif hasattr(response.choices[0].message, 'reasoning_content'):
            thinking = response.choices[0].message.reasoning_content
        elif "<think>" in answer and "</think>" in answer:
            thinking_start = answer.find("<think>") + 7
            thinking_end = answer.find("</think>")
            thinking = answer[thinking_start:thinking_end].strip()
            answer = answer[thinking_end + 8:].strip()
        
        # Filter out garbage responses (like changelogs)
        garbage_indicators = ["初始化项目", "github.com", "commit", "c0a0a0a", "1.0.0 (2023", "### Features"]
        
        if any(indicator in thinking for indicator in garbage_indicators):
            thinking = "The model generated irrelevant content instead of solving the problem. This appears to be a model issue."
        
        if any(indicator in answer for indicator in garbage_indicators):
            answer = "The model generated irrelevant output (appears to be a changelog) instead of solving the dinner party problem. This may be due to model confusion. Please try running the cell again or switch to a different model."
        
        # Ensure we have actual content
        if len(answer.strip()) == 0:
            answer = "The model did not generate a proper response. Please try again."
        
        # Calculate metrics
        metrics = {
            "response_time": round(end_time - start_time, 2),
            "thinking_length": len(thinking) if thinking else 0,
            "answer_length": len(answer),
            "approaches_count": answer.lower().count("approach") if answer else 0
        }
        
        return {
            "thinking": thinking if thinking else "No explicit thinking process detected",
            "answer": answer,
            "metrics": metrics
        }
    except Exception as e:
        return {"error": str(e)}

In [ ]:
# Complex problem to test different planning approaches
planning_problem = """
You are planning a dinner party for 8 people with the following constraints:
- Two guests are vegetarian
- One guest has a severe nut allergy
- One guest doesn't eat gluten
- You have a budget of $120 for all food and drinks
- You need to prepare an appetizer, main course, and dessert
- You have only 4 hours to prepare everything
- Your kitchen is small with only 4 burners and one oven

Create a detailed plan for the dinner party that satisfies all constraints.
"""

# Test each approach
print("Testing different planning and reasoning approaches on the same problem...\n")

# Chain of Thought
cot_result = call_cot_model(planning_problem)
display_comparison_result("CHAIN OF THOUGHT APPROACH", cot_result)

# ReAct
react_result = call_react_model(planning_problem)
display_comparison_result("REACT (REASONING + ACTING) APPROACH", react_result)

# Tree of Thoughts
tot_result = call_tot_model(planning_problem)
display_comparison_result("TREE OF THOUGHTS APPROACH", tot_result)

In [ ]:
# Comparative metrics visualization
def compare_approaches():
    approaches = ["Chain of Thought", "ReAct", "Tree of Thoughts"]
    results = [cot_result, react_result, tot_result]
    
    # Filter out results with errors
    valid_approaches = []
    valid_results = []
    for approach, result in zip(approaches, results):
        if "error" not in result:
            valid_approaches.append(approach)
            valid_results.append(result)
    
    if not valid_results:
        print("No valid results to compare")
        return
    
    # Create comparison dataframe
    metrics = {}
    for metric in valid_results[0]["metrics"].keys():
        metrics[metric] = [result["metrics"].get(metric, 0) for result in valid_results]
    
    df = pd.DataFrame(metrics, index=valid_approaches)
    
    # Visualization
    fig, axes = plt.subplots(1, len(df.columns), figsize=(15, 5))
    
    for i, column in enumerate(df.columns):
        df[column].plot(kind='bar', ax=axes[i], title=column)
        axes[i].set_ylabel(column)
    
    plt.tight_layout()
    plt.show()
    
    # Display comparison table
    print("\n" + "=" * 80)
    print("📊 COMPARATIVE ANALYSIS")
    print("=" * 80)
    print(df)
    
    # Qualitative analysis
    print("\n" + "=" * 80)
    print("🔍 QUALITATIVE COMPARISON")
    print("=" * 80)
    print("""
    Chain of Thought (CoT):
    - Strengths: Linear reasoning, direct path to solution
    - Weaknesses: May get stuck in a single approach
    - Best for: Well-defined problems with clear solution paths
    
    ReAct (Reasoning + Acting):
    - Strengths: Combines reasoning with simulated actions
    - Weaknesses: May require more computation time
    - Best for: Problems requiring information gathering and step-by-step verification
    
    Tree of Thoughts (ToT):
    - Strengths: Explores multiple solution pathways, can backtrack if needed
    - Weaknesses: More complex, potentially redundant exploration
    - Best for: Complex problems with multiple valid approaches or potential dead ends
    """)

# Run the comparison
compare_approaches()

In [ ]:
def test_custom_problem():
    """Let user try different approaches with a custom problem"""
    
    custom_problem = input("Enter your own problem to test different planning approaches:\n")
    
    approach = input("\nSelect approach (1=CoT, 2=ReAct, 3=ToT, 4=All): ")
    
    if approach == "1" or approach == "4":
        cot_custom = call_cot_model(custom_problem)
        display_comparison_result("CHAIN OF THOUGHT - CUSTOM PROBLEM", cot_custom)
    
    if approach == "2" or approach == "4":
        react_custom = call_react_model(custom_problem)
        display_comparison_result("REACT - CUSTOM PROBLEM", react_custom)
    
    if approach == "3" or approach == "4":
        tot_custom = call_tot_model(custom_problem)
        display_comparison_result("TREE OF THOUGHTS - CUSTOM PROBLEM", tot_custom)

# Uncomment to run the interactive test
#test_custom_problem()

## Key Takeaways: Planning & Reasoning Approaches

This extension demonstrated three different planning and reasoning approaches using DeepSeek models:

1. **Chain of Thought (CoT)**
   - Linear step-by-step reasoning
   - Provides clear explanations for each step
   - Works well for straightforward problems

2. **ReAct (Reasoning + Acting)**
   - Combines reasoning with simulated actions
   - Structures thinking into Thought-Action-Observation cycles
   - Effective for problems requiring information gathering or verification

3. **Tree of Thoughts (ToT)**
   - Explores multiple solution paths simultaneously
   - Evaluates different approaches and selects the most promising
   - Can backtrack from unproductive paths
   - Useful for complex problems with multiple viable approaches

## When to Use Each Approach:

- **Use CoT** when you need a straightforward solution path with clear explanations
- **Use ReAct** when your task involves tools, information gathering, or environmental interaction
- **Use ToT** when facing complex problems with multiple potential approaches or dead ends

## Choosing the Right DeepSeek Model:

- **DeepSeek R1 (deepseek/deepseek-r1)**: Best for complex reasoning tasks where quality matters most
- **DeepSeek R1 Distill Models**: Good balance of performance and cost for production use
- **DeepSeek Chat**: Fast responses for simpler tasks without reasoning visibility

These approaches can be combined or customized to create hybrid strategies for different tasks.

## Conclusion

This notebook demonstrated how DeepSeek's thinking models provide a window into AI reasoning processes, showcasing fundamental agentic capabilities without complex frameworks.

### Key Learning Points:

1. **Transparency in AI Reasoning**: 
   - Unlike traditional LLMs that only show final answers, thinking models reveal the step-by-step reasoning process
   - This transparency helps us understand, verify, and learn from AI decision-making

2. **Core Agentic Capabilities**:
   - **Planning**: Breaking down complex problems into manageable steps
   - **Reasoning**: Logical thinking through intermediate steps
   - **Reflection**: Self-evaluation and error detection
   - **Revision**: Adapting approaches based on reflection
   - **Production**: Generating final, refined outputs

3. **Practical Applications**:
   - **Problem Solving**: Mathematical calculations, logic puzzles
   - **Creative Tasks**: Story writing with iterative improvement
   - **Planning Tasks**: Multi-constraint optimization (dinner party example)
   - **Multi-Agent Simulation**: Complex collaborative scenarios

4. **Different Reasoning Approaches**:
   - **Chain of Thought**: Linear, step-by-step reasoning
   - **ReAct**: Combining reasoning with simulated actions
   - **Tree of Thoughts**: Exploring multiple solution paths

5. **Cost-Effective AI Development**:
   - DeepSeek models offer powerful reasoning capabilities at a fraction of the cost of frontier models
   - The paid models (like deepseek/deepseek-r1) provide reliable performance for educational and production use

### Next Steps:

1. **Experiment with Different Problems**: Try the models on your own complex reasoning tasks
2. **Compare Approaches**: Test which reasoning approach works best for different problem types
3. **Build Applications**: Use these reasoning capabilities as building blocks for more complex AI systems
4. **Explore Frameworks**: Once comfortable with basic reasoning, explore agent frameworks like LangChain or AutoGen

This foundation in understanding AI reasoning will help you build more sophisticated, reliable, and transparent AI systems.